# TensorFlow Tutorial - 5. RNN

본 문서는 TensorFlow 를 사용하여 Deep Learning을 구현하기 위한 기초적인 실습 자료이다. 첫 번째 파트에서는 tensorflow에 대한 기본적인 설명과 deep learning 예제를 다루어보고, 두 번째 파트에서는 오픈소스를 활용한 Deep Reinforcement Learning 을 실습해보는 시간을 갖는다. 마지막으로는 TensorFlow로 구현되고 공개된 여러 오픈소스를 둘러본다.

The code and comments are written by Dong-Hyun Kwak <imcomking@gmail.com>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.




# Agenda
## [1. Tensorflow Intro](../TensorflowIntro/TensorflowIntro.ipynb)
## [2. Basic](../Basic/Basic.ipynb)
## [3. Gradient Descent](../GradientDescent/GradientDescent.ipynb)
## [4. MLP](../MLP/MLP.ipynb)
## [5. RNN](../RNN/RNN.ipynb)
## [6. CNN](../CNN/CNN.ipynb)
## [7. Reinforcemet Learning](../ReinforcemetLearning/ReinforcemetLearning.ipynb)


### Recurrent Neural Networks
지금까지 배운 MLP, CNN과 더불어 Deep Learning에서 쓰이는 가장 강력한 모델로 Recurrent Neural Networks를 빼놓을 수 없다. 마지막으로 이 알고리즘 또한 익혀보자.


Recurrent Neural Networks, 이하 RNN는 다음과 같은 구조를 가진 모델이다. RNN은 자기자신을 향하는 weight를 이용해 데이터간의 시간관계를 학습할 수 있다. 이러한 문제들을 시계열 학습이라고 부르며, 기존에 널리 쓰이던 Hidden Markov Model을 뉴럴넷을 이용해 구현했다고 볼 수 있다.



<img src="SimpleRNN01.png">
(출처: https://raw.githubusercontent.com/peterroelants/peterroelants.github.io/master/notebooks/RNN_implementation/img/SimpleRNN01.png)

위의 구조는 1개의 Recurrent weight를 가진 hidden node이다. 이러한 hidden node들이 여러개를 모여 1개의 RNN layer를 형성하고, 이것이 다시 deep 하게 쌓이는 모델 또한 가능하다.(그러나 RNN은 deep 하게 쌓을 경우 학습이 쉽지 않다.)

RNN의 경우 MLP나 CNN에 비해서 구현이 다소 복잡하다. 따라서 RNN은 skflow 라는 TensorFlow 공식 wrapping library를 활용해서 구현해보자.

https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/learn/python/learn

Skflow는 사용자가 최소한의 노력으로, 매우 추상화된 함수들을 사용해 deep learning 모델을 구축하고 학습할 수 있게 도와주는 역할을 한다.



<img src="./word_embeddings_colah.png" width='600'>
(출처: http://sebastianruder.com/word-embeddings-1/)




In [1]:
# Recurrent Neural Network
import numpy as np
import tensorflow as tf

char_rdic = list('helo')  # id -> char
char_dic = {w: i for i, w in enumerate(char_rdic)}  # char -> id

x_data = np.array([
    [1, 0, 0, 0],  # h
    [0, 1, 0, 0],  # e
    [0, 0, 1, 0],  # l
    [0, 0, 1, 0],  # l
],
    dtype='f')

sample = [char_dic[c] for c in "hello"]  # to index

# Configuration
char_vocab_size = len(char_dic)
rnn_size = char_vocab_size  # 1 hot coding (one of 4)
time_step_size = 4  # 'hell' -> predict 'ello'
batch_size = 1  # one sample

# RNN model
#rnn_cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
rnn_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
state = tf.zeros([batch_size, rnn_cell.state_size])
X_split = tf.split(axis=0, num_or_size_splits=time_step_size, value=x_data)
#outputs, state = tf.nn.rnn(rnn_cell, X_split, state)
outputs, state = tf.contrib.rnn.static_rnn(rnn_cell, X_split, state)


# logits: list of 2D Tensors of shape [batch_size x num_decoder_symbols]
# targets: list of 1D batch-sized int32 Tensors of the same length as logits.
# weights: list of 1D batch-sized float-Tensors of the same length as logits.
logits = tf.reshape(tf.concat(axis=1, values=outputs), [-1, rnn_size])
targets = tf.reshape(sample[1:], [-1])
weights = tf.ones([time_step_size * batch_size])

loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()
    for i in range(100):
        sess.run(train_op)
        result = sess.run(tf.arg_max(logits, 1))
        print("%r, %r" % (result, [char_rdic[t] for t in result]))

array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 3, 1, 3]), ['e', 'o', 'e', 'o']
array([1, 2, 1, 3]), ['e', 'l', 'e', 'o']
array([1, 2, 1, 3]), ['e', 'l', 'e', 'o']
array([1, 2, 1, 3]), ['e', 'l', 'e', 'o']
array([1, 2, 1, 3]), ['e', 'l', 'e